In [2]:
import numpy as np                
import pandas as pd
import os

def file_name( user_dir):
        file_list = list()
        for root, dirs, files in os.walk(user_dir):
            for file in files:
                # if os.path.splitext(file)[1] == '.txt':
                if file.split('.')[-1]=='csv':
                    file_list.append(os.path.join(root, file))
        return file_list

path="car"     #此处的路径为存放excel的根目录
csv=file_name(path)         #得到一个列表，里面为10个excel的存放地址
print(csv)

['car\\AA00002.csv', 'car\\AB00006.csv', 'car\\AD00003.csv', 'car\\AD00013.csv', 'car\\AD00053.csv', 'car\\AD00083.csv', 'car\\AD00419.csv', 'car\\AF00098.csv', 'car\\AF00131.csv', 'car\\AF00373.csv']


In [9]:
import math

class cal_distance(object):                          #功能为输入参数为两点的经纬度计算两点的球面距离

    def __init__(self,**kwargs):
        self.lat1 = kwargs.get('lat1')
        self.lon1 = kwargs.get('lon1')
        self.lat2 = kwargs.get('lat2')
        self.lon2 = kwargs.get('lon2')    
    def twopoint_distance(self):                     #距离计算功能函数，输出两点距离，单位为千米(公里)。
        R=6371.393                                   #地球半径（假设均匀球体），单位：公里
        dlat=self.deg2rad(self.lat2-self.lat1)       #半正失公式
        dlon=self.deg2rad(self.lon2-self.lon1)
        a=math.sin(dlat/2)**2+math.cos(self.deg2rad(self.lat1))*math.cos(self.deg2rad(self.lat2))*math.sin(dlon/2)**2
        c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
        return R*c    
    def deg2rad(self,deg):                          #度数转弧度函数
        return deg*(math.pi/180)

    

def run(data):
    realDistance=[]                                
    for i in range(len(data["lat"])-1):
        point1_lat = data["lat"][i]
        point2_lat = data["lat"][i+1]
        point1_lon = data["lng"][i]
        point2_lon = data["lng"][i+1]
        Distance = cal_distance(lat1=point1_lat,lon1=point1_lon,lat2=point2_lat,lon2=point2_lon)
        distance = Distance.twopoint_distance()
        realDistance.append(distance)
    return realDistance

    
def main():
    print("未去除熄火状态点时")
    for i in range(len(csv)):
        data=pd.read_csv(csv[i])
        realDistance=[]
        realDistance=run(data)
        maxDistance=0.033                            #正常速度内，以单位时间（1s）计算的最大行驶距离
        t1=[]                                        #未考虑GPS记录时两个记录条之间时间间隔非1s的情况
        for j in range(len(realDistance)):           
            if realDistance[j]>maxDistance:
                t1.append(1)
            else:
                t1.append(0)
    #    print("realDistance:","\n",realDistance)
    #    print("t1:","\n",t1)
        count=0
        for k in range(len(t1)):
            if t1[k]==1:
                count+=1
        print(csv[i],"漂移点个数： ",count)
    
main()

未去除熄火状态点时
car\AA00002.csv 漂移点个数：  4004
car\AB00006.csv 漂移点个数：  24918
car\AD00003.csv 漂移点个数：  9545
car\AD00013.csv 漂移点个数：  315
car\AD00053.csv 漂移点个数：  6643
car\AD00083.csv 漂移点个数：  137
car\AD00419.csv 漂移点个数：  5725
car\AF00098.csv 漂移点个数：  943
car\AF00131.csv 漂移点个数：  411
car\AF00373.csv 漂移点个数：  152


In [8]:
print("去除熄火状态点时")
for i in range(len(csv)):
    data2=pd.read_csv(csv[i])
    for n in range(len(data2["acc_state"])):
        if data2["acc_state"][n]==0:
            data2.drop([n])
    realDistance2=[]
    realDistance2=run(data2)
    maxDistance=0.033                            
    t2=[]                                        
    for j in range(len(realDistance2)):           
        if realDistance2[j]>maxDistance:
            t2.append(1)
        else:
            t2.append(0)
    count2=0
    for k in range(len(t2)):
        if t2[k]==1:
            count2+=1
    print(csv[i],"漂移点个数： ",count2)

去除熄火状态点时
car\AA00002.csv 漂移点个数：  4004
car\AB00006.csv 漂移点个数：  24918
car\AD00003.csv 漂移点个数：  9545
car\AD00013.csv 漂移点个数：  315
car\AD00053.csv 漂移点个数：  6643
car\AD00083.csv 漂移点个数：  137
car\AD00419.csv 漂移点个数：  5725
car\AF00098.csv 漂移点个数：  943
car\AF00131.csv 漂移点个数：  411
car\AF00373.csv 漂移点个数：  152
